# Load libraries

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import random
import pickle
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision.models
from torchvision import transforms

In [4]:
# https://pytorch.org/docs/stable/notes/randomness.html

seed = 20200701
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
from torch.utils.tensorboard import SummaryWriter

In [6]:
this_hparams = {
    'slno': 11,
    'weights': 'imagenet',
    'dataaug': 'mixup',
    'comments': 'no-metadata'
}

In [7]:
import shutil
this_filename = f"{this_hparams['slno']}-{this_hparams['weights']}-{this_hparams['dataaug']}-{this_hparams['comments']}"
this_log_dir = f"./tfboard_out/{this_filename}"
shutil.rmtree(this_log_dir, ignore_errors=True)
tbwriter = SummaryWriter(this_log_dir)

In [8]:
this_filename

'11-imagenet-mixup-no-metadata'

# Prepare the data

In [9]:
def prepare_data(df, unknown_to_known):
    df = df.reset_index()
    df['slno'] = df.assign(slno=1).groupby('audio_filename')['slno'].cumsum()
    df.set_index(['audio_filename', 'slno'], inplace=True)

    df_unknown = df.copy().loc[:, list(unknown_to_known.keys())]
    df.drop(columns=list(unknown_to_known.keys()), inplace=True)

    y_mask = df.copy()
    y_mask.loc[:, :] = 1
    for unknown, known in unknown_to_known.items():
        y_mask.loc[
            df_unknown[unknown] > 0.5,
            known
        ] = 0

    df = df.swaplevel(i=1, j=0, axis=0).sort_index()

    y_mask = y_mask.swaplevel(i=1, j=0, axis=0).sort_index()

    y = np.concatenate([
        df.loc[[1], :].values[..., np.newaxis],
        df.loc[[2], :].values[..., np.newaxis],
        df.loc[[3], :].values[..., np.newaxis]
    ], axis=2)

    y_mask = np.concatenate([
        y_mask.loc[[1], :].values[..., np.newaxis],
        y_mask.loc[[2], :].values[..., np.newaxis],
        y_mask.loc[[3], :].values[..., np.newaxis]
    ], axis=2)

    X = np.concatenate([
        np.expand_dims(np.load('./data/logmelspec/{}.npy'.format(x)).T[:635, :], axis=0)
        for x in df.loc[[1], :].reset_index(1).audio_filename.tolist()])
    X = np.expand_dims(X, axis=1)

    return X, y, y_mask


In [10]:
with open('./data/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

In [11]:
unknown_to_known = (
    pd.merge(metadata['taxonomy_df'].loc[lambda x: x.fine_id == 'X', ['fine', 'coarse']],
             metadata['taxonomy_df'].loc[lambda x: x.fine_id != 'X', ['fine', 'coarse']],
             on='coarse', how='inner')
    .drop(columns='coarse')
    .groupby('fine_x')['fine_y']
    .apply(lambda x: list(x)).to_dict())
known_labels = metadata['taxonomy_df'].loc[lambda x: x.fine_id != 'X'].fine.tolist()

In [12]:
train_df = pd.concat([metadata['coarse_train'], metadata['fine_train']], axis=1, sort=True)
valid_df = pd.concat([metadata['coarse_valid'], metadata['fine_valid']], axis=1, sort=True)

In [13]:
train_X, train_y, train_y_mask = prepare_data(train_df, unknown_to_known)
valid_X, valid_y, valid_y_mask = prepare_data(valid_df, unknown_to_known)

In [14]:
train_X.shape
train_y.shape
train_y_mask.shape
valid_X.shape
valid_y.shape
valid_y_mask.shape

(13538, 1, 635, 128)

(13538, 31, 3)

(13538, 31, 3)

(4308, 1, 635, 128)

(4308, 31, 3)

(4308, 31, 3)

In [15]:
channel_means = train_X.reshape(-1, 128).mean(axis=0).reshape(1, 1, 1, -1)
channel_stds = train_X.reshape(-1, 128).std(axis=0).reshape(1, 1, 1, -1)
train_X = (train_X - channel_means) / channel_stds
valid_X = (valid_X - channel_means) / channel_stds
#np.save('data/channel_means.npy', channel_means)
#np.save('data/channel_stds.npy', channel_stds)

In [16]:
import gc
gc.collect()

1000

# Prepare the PyTorch datasets

In [17]:
class AudioDataset(Dataset):

    def __init__(self, X, y, weights, transform=None):
        self.X = X
        self.y = y
        self.weights = weights

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        sample = self.X[idx, ...]
        return sample, self.y[idx, ...], self.weights[idx, ...]

In [18]:
train_dataset = AudioDataset(torch.Tensor(train_X),
                             torch.Tensor(train_y),
                             torch.Tensor(train_y_mask),
                             None)
valid_dataset = AudioDataset(torch.Tensor(valid_X),
                             torch.Tensor(valid_y),
                             torch.Tensor(valid_y_mask),
                             None)

In [19]:
val_loader = DataLoader(valid_dataset, 128, shuffle=False,
                        num_workers=2, drop_last=False, pin_memory=True)
train_loader_1 = DataLoader(train_dataset, 128, shuffle=True,
                            num_workers=2, drop_last=True, pin_memory=True)
train_loader_2 = DataLoader(train_dataset, 128, shuffle=True,
                            num_workers=2, drop_last=True, pin_memory=True)

# Define the model

In [20]:
cuda = True
device = torch.device('cuda:0' if cuda else 'cpu')
print('Device: ', device)

Device:  cuda:0


In [21]:
class Model(nn.Module):

    def __init__(self, num_classes):

        super().__init__()

        self.bw2col = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.Conv2d(1, 10, 1, padding=0), nn.ReLU(),
            nn.Conv2d(10, 3, 1, padding=0), nn.ReLU())

        self.mv2 = torchvision.models.mobilenet_v2(pretrained=True)

        self.final1 = nn.BatchNorm1d(1280)
        self.dp = nn.Dropout(0.8)
        self.final2 = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.bw2col(x)
        x = self.mv2.features(x)
        x = x.max(dim=-1)[0].max(dim=-1)[0]
        x = self.final1(x)
        x = self.dp(x)
        x = self.final2(x)
        return x

In [22]:
model = Model(31).to(device)

In [23]:
model

Model(
  (bw2col): Sequential(
    (0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(1, 10, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
    (3): Conv2d(10, 3, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
  )
  (mv2): MobileNetV2(
    (features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): Batc

# Bias training

In [24]:
optimizer = optim.Adam([model.final2.bias], lr=1, amsgrad=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)
criterion = nn.BCEWithLogitsLoss(reduction='none')

In [25]:
num_bias_training = 2
train_loss_hist = []
valid_loss_hist = []
lowest_val_loss = np.inf
epochs_without_new_lowest = 0
train_batch_slno=0

for i in range(num_bias_training):
    print('Epoch: ', i)

    this_epoch_train_loss = 0
    for i1, i2 in zip(train_loader_1, train_loader_2):

        # mixup the inputs ---------
        alpha = 1
        mixup_vals = np.random.beta(alpha, alpha, i1[0].shape[0])

        lam = torch.Tensor(mixup_vals.reshape(mixup_vals.shape[0], 1, 1, 1))
        inputs = (lam * i1[0]) + ((1 - lam) * i2[0])

        lam = torch.Tensor(mixup_vals.reshape(mixup_vals.shape[0], 1, 1))
        labels = (lam * i1[1]) + ((1 - lam) * i2[1])
        masks = (lam * i1[2]) + ((1 - lam) * i2[2])
        # mixup ends ----------

        inputs = inputs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            model = model.train()
            outputs = model(inputs)
            # calculate loss for each set of annotations
            loss_0 = criterion(outputs, labels[:, :, 0]) * masks[:, :, 0]
            loss_1 = criterion(outputs, labels[:, :, 1]) * masks[:, :, 1]
            loss_2 = criterion(outputs, labels[:, :, 2]) * masks[:, :, 2]
            loss = (loss_0.sum() + loss_1.sum() + loss_2.sum()) / masks.sum()
            loss.backward()
            optimizer.step()
            this_epoch_train_loss += loss.detach().cpu().numpy()
            tbwriter.add_scalar('Batch/Loss/Train',
                                loss.detach().cpu().numpy(),
                                train_batch_slno)
            train_batch_slno += 1

    this_epoch_valid_loss = 0
    for inputs, labels, masks in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(False):
            model = model.eval()
            outputs = model(inputs)
            loss_0 = criterion(outputs, labels[:, :, 0]) * masks[:, :, 0]
            loss_1 = criterion(outputs, labels[:, :, 1]) * masks[:, :, 1]
            loss_2 = criterion(outputs, labels[:, :, 2]) * masks[:, :, 2]
            loss = (loss_0.sum() + loss_1.sum() + loss_2.sum()) / masks.sum()
            this_epoch_valid_loss += loss.detach().cpu().numpy()

    this_epoch_train_loss /= len(train_loader_1)
    this_epoch_valid_loss /= len(val_loader)

    train_loss_hist.append(this_epoch_train_loss)
    valid_loss_hist.append(this_epoch_valid_loss)

    if this_epoch_valid_loss < lowest_val_loss:
        lowest_val_loss = this_epoch_valid_loss
        torch.save(model.state_dict(), f"./models/{this_filename}")
        epochs_without_new_lowest = 0
    else:
        epochs_without_new_lowest += 1

    if epochs_without_new_lowest >= 25:
        break

    print(this_epoch_train_loss, this_epoch_valid_loss)
    tbwriter.add_scalar('Epoch/Loss/Train', this_epoch_train_loss, i)
    tbwriter.add_scalar('Epoch/Loss/Valid', this_epoch_valid_loss, i)
    tbwriter.add_scalar('Epoch/lr', optimizer.param_groups[0]['lr'], i)
    tbwriter.flush()
    
    scheduler.step(this_epoch_valid_loss)

Epoch:  0
0.25291377235026585 0.21144344131736195
Epoch:  1
0.23212459129946572 0.2062930249992539


# Training

In [26]:
optimizer = optim.Adam(model.parameters(), lr=0.0005, amsgrad=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)
criterion = nn.BCEWithLogitsLoss(reduction='none')

In [27]:
epochs = 100
for i in range(num_bias_training, epochs):
    print('Epoch: ', i)

    this_epoch_train_loss = 0
    for i1, i2 in zip(train_loader_1, train_loader_2):

        # mixup the inputs ---------
        alpha = 1
        mixup_vals = np.random.beta(alpha, alpha, i1[0].shape[0])

        lam = torch.Tensor(mixup_vals.reshape(mixup_vals.shape[0], 1, 1, 1))
        inputs = (lam * i1[0]) + ((1 - lam) * i2[0])

        lam = torch.Tensor(mixup_vals.reshape(mixup_vals.shape[0], 1, 1))
        labels = (lam * i1[1]) + ((1 - lam) * i2[1])
        masks = (lam * i1[2]) + ((1 - lam) * i2[2])
        # mixup ends ----------

        inputs = inputs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            model = model.train()
            outputs = model(inputs)
            # calculate loss for each set of annotations
            loss_0 = criterion(outputs, labels[:, :, 0]) * masks[:, :, 0]
            loss_1 = criterion(outputs, labels[:, :, 1]) * masks[:, :, 1]
            loss_2 = criterion(outputs, labels[:, :, 2]) * masks[:, :, 2]
            loss = (loss_0.sum() + loss_1.sum() + loss_2.sum()) / masks.sum()
            loss.backward()
            optimizer.step()
            this_epoch_train_loss += loss.detach().cpu().numpy()
            tbwriter.add_scalar('Batch/Loss/Train',
                                loss.detach().cpu().numpy(),
                                train_batch_slno)
            train_batch_slno += 1

    this_epoch_valid_loss = 0
    for inputs, labels, masks in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(False):
            model = model.eval()
            outputs = model(inputs)
            loss_0 = criterion(outputs, labels[:, :, 0]) * masks[:, :, 0]
            loss_1 = criterion(outputs, labels[:, :, 1]) * masks[:, :, 1]
            loss_2 = criterion(outputs, labels[:, :, 2]) * masks[:, :, 2]
            loss = (loss_0.sum() + loss_1.sum() + loss_2.sum()) / masks.sum()
            this_epoch_valid_loss += loss.detach().cpu().numpy()

    this_epoch_train_loss /= len(train_loader_1)
    this_epoch_valid_loss /= len(val_loader)

    train_loss_hist.append(this_epoch_train_loss)
    valid_loss_hist.append(this_epoch_valid_loss)

    if this_epoch_valid_loss < lowest_val_loss:
        lowest_val_loss = this_epoch_valid_loss
        torch.save(model.state_dict(), f"./models/{this_filename}")
        epochs_without_new_lowest = 0
    else:
        epochs_without_new_lowest += 1

    if epochs_without_new_lowest >= 25:
        break

    print(this_epoch_train_loss, this_epoch_valid_loss)
    tbwriter.add_scalar('Epoch/Loss/Train', this_epoch_train_loss, i)
    tbwriter.add_scalar('Epoch/Loss/Valid', this_epoch_valid_loss, i)
    tbwriter.add_scalar('Epoch/lr', optimizer.param_groups[0]['lr'], i)
    tbwriter.flush()
    
    scheduler.step(this_epoch_valid_loss)

Epoch:  2
0.18847813166323163 0.14792317239677205
Epoch:  3
0.16612780931450072 0.13788410670617046
Epoch:  4
0.15869193091278985 0.1357972718337003
Epoch:  5
0.15502896592730567 0.1319239766282194
Epoch:  6
0.1531985055832636 0.1297704634421012
Epoch:  7
0.15156994476204827 0.1314088053563062
Epoch:  8
0.1501519081138429 0.12867636014433467
Epoch:  9
0.14978211919466655 0.12781126599978
Epoch:  10
0.14870465497175853 0.12917187108713038
Epoch:  11
0.14841742146582831 0.12794245976735563
Epoch:  12
0.14827138753164382 0.12789632533403003
Epoch:  13
0.14727834463119507 0.12984971044694676
Epoch:  14
0.14653282463550568 0.1278854996404227
Epoch:  15
0.14531180219990866 0.12820994349963524
Epoch    14: reducing learning rate of group 0 to 5.0000e-05.
Epoch:  16
0.14426287540367672 0.1249823879231425
Epoch:  17
0.14373313046637035 0.12471026313655517
Epoch:  18
0.14306478926113675 0.12456478068933767
Epoch:  19
0.14311998387177785 0.12445246209116544
Epoch:  20
0.1424712835323243 0.1251741

# Evaluation metrics

In [28]:
model.load_state_dict(torch.load(
    f"./models/{this_filename}"
))

<All keys matched successfully>

In [29]:
this_metrics = {}

---

In [30]:
this_epoch_valid_loss = 0
val_preds = []
for inputs, labels, masks in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(False):
            model = model.eval()
            outputs = model(inputs)
            loss_0 = criterion(outputs, labels[:, :, 0]) * masks[:, :, 0]
            loss_1 = criterion(outputs, labels[:, :, 1]) * masks[:, :, 1]
            loss_2 = criterion(outputs, labels[:, :, 2]) * masks[:, :, 2]
            loss = (loss_0.sum() + loss_1.sum() + loss_2.sum()) / masks.sum()
            this_epoch_valid_loss += loss.detach().cpu().numpy()
            val_preds.append(outputs.detach().cpu().numpy())
this_epoch_valid_loss /= len(val_loader)
val_preds = np.concatenate(val_preds, axis=0)

In [31]:
this_epoch_valid_loss, val_preds.shape

(0.12423181906342506, (4308, 31))

In [32]:
this_metrics['valid_loss'] = this_epoch_valid_loss

In [33]:
train_nodataaug_dataset = AudioDataset(torch.Tensor(train_X),
                                       torch.Tensor(train_y),
                                       torch.Tensor(train_y_mask),
                                       None)
train_nodataaug_loader = DataLoader(train_nodataaug_dataset, 64, shuffle=False)

this_epoch_train_nodataaug_loss = 0
train_preds = []
for inputs, labels, masks in train_nodataaug_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(False):
            model = model.eval()
            outputs = model(inputs)
            loss_0 = criterion(outputs, labels[:, :, 0]) * masks[:, :, 0]
            loss_1 = criterion(outputs, labels[:, :, 1]) * masks[:, :, 1]
            loss_2 = criterion(outputs, labels[:, :, 2]) * masks[:, :, 2]
            loss = (loss_0.sum() + loss_1.sum() + loss_2.sum()) / masks.sum()
            this_epoch_train_nodataaug_loss += loss.detach().cpu().numpy()
            train_preds.append(outputs.detach().cpu().numpy())
this_epoch_train_nodataaug_loss /= len(train_nodataaug_loader)
train_preds = np.concatenate(train_preds, axis=0)

In [34]:
this_epoch_train_nodataaug_loss, train_preds.shape

(0.1083423182157413, (13538, 31))

In [35]:
this_metrics['train_loss'] = this_epoch_train_nodataaug_loss

---

In [36]:
our_cols = ['1_engine', '2_machinery-impact', '3_non-machinery-impact',
            '4_powered-saw', '5_alert-signal', '6_music', '7_human-voice', '8_dog',
            '1-1_small-sounding-engine', '1-2_medium-sounding-engine',
            '1-3_large-sounding-engine', '2-1_rock-drill', '2-2_jackhammer',
            '2-3_hoe-ram', '2-4_pile-driver', '3-1_non-machinery-impact',
            '4-1_chainsaw', '4-2_small-medium-rotating-saw',
            '4-3_large-rotating-saw', '5-1_car-horn', '5-2_car-alarm', '5-3_siren',
            '5-4_reverse-beeper', '6-1_stationary-music', '6-2_mobile-music',
            '6-3_ice-cream-truck', '7-1_person-or-small-group-talking',
            '7-2_person-or-small-group-shouting', '7-3_large-crowd',
            '7-4_amplified-speech', '8-1_dog-barking-whining']
cols_for_out = [
    "audio_filename", "1-1_small-sounding-engine",
    "1-2_medium-sounding-engine", "1-3_large-sounding-engine",
    "2-1_rock-drill",
    "2-2_jackhammer", "2-3_hoe-ram", "2-4_pile-driver",
    "3-1_non-machinery-impact",
    "4-1_chainsaw", "4-2_small-medium-rotating-saw",
    "4-3_large-rotating-saw",
    "5-1_car-horn", "5-2_car-alarm", "5-3_siren", "5-4_reverse-beeper",
    "6-1_stationary-music",
    "6-2_mobile-music", "6-3_ice-cream-truck",
    "7-1_person-or-small-group-talking",
    "7-2_person-or-small-group-shouting", "7-3_large-crowd",
    "7-4_amplified-speech",
    "8-1_dog-barking-whining", "1_engine", "2_machinery-impact",
    "3_non-machinery-impact", "4_powered-saw", "5_alert-signal",
    "6_music", "7_human-voice", "8_dog"]

---

In [37]:
val_df = pd.DataFrame((1 / (1 + np.exp(-val_preds))), columns = our_cols)
val_df['audio_filename'] = pd.Series(
    sorted(set(metadata['coarse_valid'].index.tolist())),
    index=val_df.index)
val_df = val_df.loc[:, cols_for_out]
val_df = val_df.loc[lambda x: x.audio_filename.isin(
    metadata['coarse_valid_gt'].index.tolist())]

In [38]:
val_df.to_csv('/tmp/val_subm.csv', index=False)

---

In [39]:
import sys
sys.path.append('./baseline_code')

In [40]:
from metrics import evaluate, macro_averaged_auprc, micro_averaged_auprc

---

In [41]:
df_dict = evaluate(
    '/tmp/val_subm.csv',
    './data/annotations.csv',
    './data/dcase-ust-taxonomy.yaml',
    'coarse'
)
micro_auprc, eval_df = micro_averaged_auprc(df_dict, return_df=True)
macro_auprc, class_auprc = macro_averaged_auprc(df_dict, return_classwise=True)
thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).to_numpy().nonzero()[0][0]

/home/sainatha/dcase2020-task5-urban-sound-tagging-with-context/.env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (47,51,52,53,58,63,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
micro_auprc, macro_auprc, eval_df['F'][thresh_0pt5_idx]
class_auprc

(0.8796673021190263, 0.7186267647260319, 0.6864197530864198)

{1: 0.8886535363418737,
 2: 0.7552799390978309,
 3: 0.6033522575128015,
 4: 0.7708179340714502,
 5: 0.9428146358247643,
 6: 0.6107426118672498,
 7: 0.9817033445768305,
 8: 0.19564985851545505}

In [43]:
this_metrics['valid_coarse_micro_auprc'] = micro_auprc
this_metrics['valid_coarse_macro_auprc'] = macro_auprc
this_metrics['valid_coarse_f1'] = eval_df['F'][thresh_0pt5_idx]
for k, v in class_auprc.items():
    this_metrics[f"valid_coarse_auprc_class_{k}"] = v

In [44]:
df_dict = evaluate(
    '/tmp/val_subm.csv',
    './data/annotations.csv',
    './data/dcase-ust-taxonomy.yaml',
    'fine'
)
micro_auprc, eval_df = micro_averaged_auprc(df_dict, return_df=True)
macro_auprc, class_auprc = macro_averaged_auprc(df_dict, return_classwise=True)
thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).to_numpy().nonzero()[0][0]

./baseline_code/metrics.py:609: UserWarning: Column not found: 1-X_engine-of-uncertain-size
  warnings.warn("Column not found: " + f)
./baseline_code/metrics.py:609: UserWarning: Column not found: 2-X_other-unknown-impact-machinery
  warnings.warn("Column not found: " + f)
./baseline_code/metrics.py:609: UserWarning: Column not found: 4-X_other-unknown-powered-saw
  warnings.warn("Column not found: " + f)
./baseline_code/metrics.py:609: UserWarning: Column not found: 5-X_other-unknown-alert-signal
  warnings.warn("Column not found: " + f)
./baseline_code/metrics.py:609: UserWarning: Column not found: 6-X_music-from-uncertain-source
  warnings.warn("Column not found: " + f)
./baseline_code/metrics.py:609: UserWarning: Column not found: 7-X_other-unknown-human-voice
  warnings.warn("Column not found: " + f)


In [45]:
micro_auprc, macro_auprc, eval_df['F'][thresh_0pt5_idx]
class_auprc

(0.7969416192318253, 0.6055958097376739, 0.5404944586530264)

{1: 0.7323197579670575,
 2: 0.5505459865539681,
 3: 0.6044451827013513,
 4: 0.5506128220289073,
 5: 0.9127783079055649,
 6: 0.3630498595711129,
 7: 0.934901657322482,
 8: 0.19611290385094654}

In [46]:
this_metrics['valid_fine_micro_auprc'] = micro_auprc
this_metrics['valid_fine_macro_auprc'] = macro_auprc
this_metrics['valid_fine_f1'] = eval_df['F'][thresh_0pt5_idx]
for k, v in class_auprc.items():
    this_metrics[f"valid_fine_auprc_class_{k}"] = v

---

In [47]:
train_nodataaug_df = pd.DataFrame((1 / (1 + np.exp(-train_preds))), columns = our_cols)
train_nodataaug_df['audio_filename'] = pd.Series(
    sorted(set(metadata['coarse_train'].index.tolist())),
    index=train_nodataaug_df.index)
train_nodataaug_df = train_nodataaug_df.loc[:, cols_for_out]
train_nodataaug_df = train_nodataaug_df.loc[lambda x: x.audio_filename.isin(
    metadata['coarse_train_gt'].index.tolist())]

In [48]:
train_nodataaug_df.to_csv('/tmp/train_nodataaug.csv', index=False)

In [49]:
df_dict = evaluate(
    '/tmp/train_nodataaug.csv',
    './data/annotations.csv',
    './data/dcase-ust-taxonomy.yaml',
    'coarse',
    True
)
micro_auprc, eval_df = micro_averaged_auprc(df_dict, return_df=True)
macro_auprc, class_auprc = macro_averaged_auprc(df_dict, return_classwise=True)
thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).to_numpy().nonzero()[0][0]

In [50]:
micro_auprc, macro_auprc, eval_df['F'][thresh_0pt5_idx]
class_auprc

(0.9248958449378661, 0.810762855001412, 0.7296587926509187)

{1: 0.9352137640260354,
 2: 0.75347508598355,
 3: 0.5740290752194888,
 4: 0.6497519841269841,
 5: 0.9415787144119145,
 6: 0.7268145161290323,
 7: 0.9631547580293488,
 8: 0.9420849420849422}

In [51]:
this_metrics['train_coarse_micro_auprc'] = micro_auprc
this_metrics['train_coarse_macro_auprc'] = macro_auprc
this_metrics['train_coarse_f1'] = eval_df['F'][thresh_0pt5_idx]
for k, v in class_auprc.items():
    this_metrics[f"train_coarse_auprc_class_{k}"] = v

In [52]:
df_dict = evaluate(
    '/tmp/train_nodataaug.csv',
    './data/annotations.csv',
    './data/dcase-ust-taxonomy.yaml',
    'fine',
    True
)
micro_auprc, eval_df = micro_averaged_auprc(df_dict, return_df=True)
macro_auprc, class_auprc = macro_averaged_auprc(df_dict, return_classwise=True)
thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).to_numpy().nonzero()[0][0]

In [53]:
micro_auprc, macro_auprc, eval_df['F'][thresh_0pt5_idx]
class_auprc

(0.8262525209177717, 0.7475369662937099, 0.5210084033613446)

{1: 0.7006648602787394,
 2: 0.7271376160527103,
 3: 0.5748306785692666,
 4: 0.4783150121607783,
 5: 0.9047076966400154,
 6: 0.7727272727272727,
 7: 0.8811460433982492,
 8: 0.9407665505226481}

In [54]:
this_metrics['train_fine_micro_auprc'] = micro_auprc
this_metrics['train_fine_macro_auprc'] = macro_auprc
this_metrics['train_fine_f1'] = eval_df['F'][thresh_0pt5_idx]
for k, v in class_auprc.items():
    this_metrics[f"train_fine_auprc_class_{k}"] = v

---

In [55]:
this_metrics = {
    f'hparams/{k}': v
    for k, v in this_metrics.items()
}

In [56]:
tbwriter.add_hparams(hparam_dict=this_hparams, metric_dict=this_metrics)

In [57]:
tbwriter.flush()